- 유저 프로파일 정의 하고
- max_clicks 조절 : 유저가 클릭한 개수 -> df_logs 만들기
    - 몇십만개 만들어
- 학습데이터 구성
    - max_history=10, num_negatives=10, max_samples_per_user=500 조절
    - 학습데이터 안에서 클릭한것의 최대 개수 10개, 유저 당 최대 샘플 500개 
    - 유저 수 늘리든 유저당 최대샘플 조절해서 학습데이터 늘려봐
- 모델 돌려봐서 평가 지표 확인

# 유저 프로파일 정의

투자 성향 유형
- 4. 공격 투자형 : 손실 있음, 단기 차익, 바이오주
- 3. 적극 투자형 : 손실 있음, 미국 증시 확인하며 유동적으로 투자
- 2. 위험 중립형 : 손실 혹은 수익, 단기 차익 & 장기 차익을 동시에 고려, 포트폴리오 구성에 신경쓰는
- 1. 안정 추구형 : 수익, 유명한거 사고 배당주 위주로
- 0. 안정형 : 그냥 아무것도 모르고 제일 유명한거 사놓는

In [163]:
user_profiles = {
    "u1": {
        "userPnl": -220000,
        "asset": 7000000,
        "investScore": 4,
        "topics": [1, 6],
        "memberStocks": [
            {"stock_id": "019170", "stock_name": "신풍제약"},
            {"stock_id": "000100", "stock_name": "유한양행"}
        ],
        "memberIndustries": [
            {"stock_id": "000100", "industry_id": "32102", "industry_name": "의약품 제조업"},
            {"stock_id": "019170", "industry_id": "32102", "industry_name": "의약품 제조업"}
        ],
    },
    "u2": {
        "userPnl": 37000,
        "asset": 2500000,
        "investScore": 1,
        "topics": [2, 7, 8],
        "memberStocks": [
            {"stock_id": "005930", "stock_name": "삼성전자"},
            {"stock_id": "006800", "stock_name": "미래에셋증권"}
        ],
        "memberIndustries": [
            {"stock_id": "005930", "industry_id": "32604", "industry_name": "통신 및 방송 장비 제조업"},
            {"stock_id": "006800", "industry_id": "116601", "industry_name": "금융 지원 서비스업"}
        ],
    },
    "u3": {
        "userPnl": 420000,
        "asset": 15000000,
        "investScore": 2,
        "topics": [1, 3, 4],
        "memberStocks": [
            {"stock_id": "012450", "stock_name": "한화에어로스페이스"},
            {"stock_id": "016360", "stock_name": "삼성증권"},
            {"stock_id": "001430", "stock_name": "세아베스틸지주"}
        ],
        "memberIndustries": [
            {"stock_id": "012450", "industry_id": "33103", "industry_name": "항공기,우주선 및 부품 제조업"},
            {"stock_id": "016360", "industry_id": "116601", "industry_name": "금융 지원 서비스업"},
            {"stock_id": "001430", "industry_id": "32401", "industry_name": "1차 철강 제조업"}
        ],
    },
    "u4": {
        "userPnl": 50000,
        "asset": 1000000,
        "investScore": 1,
        "topics": [2, 3, 5],
        "memberStocks": [
            {"stock_id": "005380", "stock_name": "현대차"},
            {"stock_id": "017670", "stock_name": "SK텔레콤"}
        ],
        "memberIndustries": [
            {"stock_id": "005380", "industry_id": "33001", "industry_name": "자동차용 엔진 및 자동차 제조업"},
            {"stock_id": "017670", "industry_id": "106102", "industry_name": "전기 통신업"}
        ],
    },
    "u5": {
        "userPnl": 12000,
        "asset": 500000,
        "investScore": 0,
        "topics": [7, 9],
        "memberStocks": [
            {"stock_id": "000660", "stock_name": "SK하이닉스"}
        ],
        "memberIndustries": [
            {"stock_id": "000660", "industry_id": "32601", "industry_name": "반도체 제조업"}
        ],
    },
    "u6": {
        "userPnl": -400000,
        "asset": 20000000,
        "investScore": 3,
        "topics": [1, 4, 5],
        "memberStocks": [
            {"stock_id": "035720", "stock_name": "카카오"},
            {"stock_id": "042700", "stock_name": "한미반도체"},
            {"stock_id": "192820", "stock_name": "코스맥스"}
        ],
        "memberIndustries": [
            {"stock_id": "035720", "industry_id": "106301", "industry_name": "자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업"},
            {"stock_id": "042700", "industry_id": "32902", "industry_name": "특수 목적용 기계 제조업"},
            {"stock_id": "192820", "industry_id": "32004", "industry_name": "기타 화학제품 제조업"}
        ],
    },
    "u7": {
        "userPnl": -190000,
        "asset": 8000000,
        "investScore": 4,
        "topics": [1, 6, 9],
        "memberStocks": [
            {"stock_id": "010130", "stock_name": "고려아연"},
            {"stock_id": "352820", "stock_name": "하이브"}
        ],
        "memberIndustries": [
            {"stock_id": "010130", "industry_id": "32402", "industry_name": "1차 비철금속 제조업"},
            {"stock_id": "352820", "industry_id": "105902", "industry_name": "오디오물 출판 및 원판 녹음업"}
        ],
    },
    "u8": {
        "userPnl": 360000,
        "asset": 6000000,
        "investScore": 2,
        "topics": [2, 8, 9],
        "memberStocks": [
            {"stock_id": "267260", "stock_name": "HD현대일렉트릭"},
            {"stock_id": "034020", "stock_name": "두산에너빌리티"},
            {"stock_id": "006360", "stock_name": "GS건설"}
        ],
        "memberIndustries": [
            {"stock_id": "267260", "industry_id": "32801", "industry_name": "전기 변환·제어 장치 제조업"},
            {"stock_id": "034020", "industry_id": "32901", "industry_name": "일반 목적용 기계 제조업"},
            {"stock_id": "006360", "industry_id": "64101", "industry_name": "건물 건설업"}
        ],
    },
    "u9": {
        "userPnl": 1000000,
        "asset": 102200000,
        "investScore": 3,
        "topics": [1, 3],
        "memberStocks": [
            {"stock_id": "003230", "stock_name": "삼양식품"},
            {"stock_id": "068270", "stock_name": "셀트리온"},
            {"stock_id": "278470", "stock_name": "에이피알"}
        ],
        "memberIndustries": [
            {"stock_id": "003230", "industry_id": "31008", "industry_name": "기타 식품 제조업"},
            {"stock_id": "068270", "industry_id": "32101", "industry_name": "기초 의약물질 제조업"},
            {"stock_id": "278470", "industry_id": "32004", "industry_name": "기타 화학제품 제조업"}
        ],
    },
    "u10": {
        "userPnl": 12200000,
        "asset": 55000000,
        "investScore": 1,
        "topics": [4, 8, 9],
        "memberStocks": [
            {"stock_id": "039490", "stock_name": "키움증권"},
            {"stock_id": "086790", "stock_name": "하나금융지주"},
            {"stock_id": "005490", "stock_name": "POSCO홀딩스"},
            {"stock_id": "005930", "stock_name": "삼성전자"}
        ],
        "memberIndustries": [
            {"stock_id": "039490", "industry_id": "116601", "industry_name": "금융 지원 서비스업"},
            {"stock_id": "086790", "industry_id": "116409", "industry_name": "기타 금융업"},
            {"stock_id": "005490", "industry_id": "32401", "industry_name": "1차 철강 제조업"},
            {"stock_id": "005930", "industry_id": "32604", "industry_name": "통신 및 방송 장비 제조업"}
        ],
    },
    "u11": {
        "userPnl": 300000,
        "asset": 4000000,
        "investScore": 0,
        "topics": [1, 3],
        "memberStocks": [
            {"stock_id": "051900", "stock_name": "LG생활건강"},
            {"stock_id": "035420", "stock_name": "NAVER"},
            {"stock_id": "000270", "stock_name": "기아"}
        ],
        "memberIndustries": [
            {"stock_id": "051900", "industry_id": "32004", "industry_name": "기타 화학제품 제조업"},
            {"stock_id": "035420", "industry_id": "106301", "industry_name": "자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업"},
            {"stock_id": "000270", "industry_id": "33001", "industry_name": "자동차 제조업"}
        ],
    },
    "u12": {
        "userPnl": 0,
        "asset": 20000000,
        "investScore": 3,
        "topics": [3, 5, 9],
        "memberStocks": [
            {"stock_id": "018880", "stock_name": "한온시스템"},
            {"stock_id": "004170", "stock_name": "신세계"},
            {"stock_id": "138040", "stock_name": "메리츠금융지주"}
        ],
        "memberIndustries": [
            {"stock_id": "018880", "industry_id": "32901", "industry_name": "일반 목적용 기계 제조업"},
            {"stock_id": "004170", "industry_id": "74701", "industry_name": "종합 소매업"},
            {"stock_id": "138040", "industry_id": "116409", "industry_name": "기타 금융업"}
        ],
    },

    "u13": {
        "userPnl": -200000,
        "asset": 3000000,
        "investScore": 0,
        "topics": [8, 9],
        "memberStocks": [
            {"stock_id": "005930", "stock_name": "삼성전자"},
            {"stock_id": "000660", "stock_name": "SK하이닉스"}
        ],
        "memberIndustries": [
            {"stock_id": "005930", "industry_id": "32601", "industry_name": "반도체 제조업"},
            {"stock_id": "000660", "industry_id": "32601", "industry_name": "반도체 제조업"}
        ]
    },
    "u14": {
        "userPnl": 1000000,
        "asset": 10000000,
        "investScore": 1,
        "topics": [1, 8],
        "memberStocks": [
        {"stock_id": "105560", "stock_name": "KB금융"},
        {"stock_id": "032640", "stock_name": "LG유플러스"}
        ],
        "memberIndustries": [
        {"stock_id": "105560", "industry_id": "64101", "industry_name": "은행업"},
        {"stock_id": "032640", "industry_id": "61201", "industry_name": "무선통신업"}
        ]
    },
    "u15": {
        "userPnl": 1500000,
        "asset": 15000000,
        "investScore": 2,
        "topics": [1, 3, 4],
        "memberStocks": [
        {"stock_id": "003550", "stock_name": "LG"},
        {"stock_id": "000270", "stock_name": "기아"},
        {"stock_id": "017670", "stock_name": "SK텔레콤"}
        ],
        "memberIndustries": [
        {"stock_id": "003550", "industry_id": "106301", "industry_name": "인터넷 정보매개 서비스업"},
        {"stock_id": "000270", "industry_id": "33001", "industry_name": "자동차 제조업"},
        {"stock_id": "017670", "industry_id": "61201", "industry_name": "무선통신업"}
        ]
    },
    "u16": {
        "userPnl": 300000,
        "asset": 7000000,
        "investScore": 2,
        "topics": [4, 6],
        "memberStocks": [
        {"stock_id": "006400", "stock_name": "삼성SDI"},
        {"stock_id": "034730", "stock_name": "SK"},
        {"stock_id": "035420", "stock_name": "NAVER"}
        ],
        "memberIndustries": [
        {"stock_id": "006400", "industry_id": "32601", "industry_name": "2차전지 제조업"},
        {"stock_id": "034730", "industry_id": "106301", "industry_name": "인터넷 정보매개 서비스업"},
        {"stock_id": "035420", "industry_id": "106301", "industry_name": "포털 및 검색엔진 서비스업"}
        ]
    },
    "u17": {
        "userPnl": 800000,
        "asset": 8000000,
        "investScore": 3,
        "topics": [5, 6, 7],
        "memberStocks": [
        {"stock_id": "068270", "stock_name": "셀트리온"},
        {"stock_id": "035720", "stock_name": "카카오"},
        {"stock_id": "091990", "stock_name": "셀트리온헬스케어"}
        ],
        "memberIndustries": [
        {"stock_id": "068270", "industry_id": "21002", "industry_name": "의약품 제조업"},
        {"stock_id": "035720", "industry_id": "106301", "industry_name": "플랫폼 및 콘텐츠 유통업"},
        {"stock_id": "091990", "industry_id": "21002", "industry_name": "바이오 의약품 유통업"}
        ]
    },
    "u18": {
        "userPnl": 1200000,
        "asset": 9000000,
        "investScore": 3,
        "topics": [6, 7, 3],
        "memberStocks": [
        {"stock_id": "066570", "stock_name": "LG전자"},
        {"stock_id": "000990", "stock_name": "DB하이텍"},
        {"stock_id": "003490", "stock_name": "대한항공"}
        ],
        "memberIndustries": [
        {"stock_id": "066570", "industry_id": "26401", "industry_name": "전자제품 제조업"},
        {"stock_id": "000990", "industry_id": "32601", "industry_name": "반도체 제조업"},
        {"stock_id": "003490", "industry_id": "51201", "industry_name": "항공 운송업"}
        ]
    },
    "u19": {
        "userPnl": 2000000,
        "asset": 20000000,
        "investScore": 4,
        "topics": [7, 6, 4, 5],
        "memberStocks": [
        {"stock_id": "247540", "stock_name": "에코프로비엠"},
        {"stock_id": "373220", "stock_name": "LG에너지솔루션"},
        {"stock_id": "196170", "stock_name": "알테오젠"}
        ],
        "memberIndustries": [
        {"stock_id": "247540", "industry_id": "32601", "industry_name": "이차전지 소재 제조업"},
        {"stock_id": "373220", "industry_id": "32601", "industry_name": "이차전지 제조업"},
        {"stock_id": "196170", "industry_id": "21002", "industry_name": "바이오 의약품 제조업"}
        ]
    },
    "u20": {
        "userPnl": 2500000,
        "asset": 25000000,
        "investScore": 4,
        "topics": [7, 6, 5, 2],
        "memberStocks": [
        {"stock_id": "096530", "stock_name": "씨젠"},
        {"stock_id": "066970", "stock_name": "엘앤에프"},
        {"stock_id": "293490", "stock_name": "카카오게임즈"}
        ],
        "memberIndustries": [
        {"stock_id": "096530", "industry_id": "21002", "industry_name": "바이오 진단키트 제조업"},
        {"stock_id": "066970", "industry_id": "32601", "industry_name": "이차전지 양극재 제조업"},
        {"stock_id": "293490", "industry_id": "106301", "industry_name": "모바일 게임업"}
        ]
    }
    }

    


In [164]:
import pandas as pd

df_raw = pd.read_csv('../../db/news_2023_2025_raw.csv')
df_meta = pd.read_csv('../../db/news_2023_2025_metadata2.csv')
df_topic = pd.read_csv('../../db/news_2023_2025_tm.csv')

In [165]:
# 1. topic 컬럼만 추출 (news_id 제외)
topic_cols = [col for col in df_topic.columns if col.startswith("topic")]

# 2. 각 행마다 가장 큰 값 가진 컬럼의 숫자 인덱스만 추출
df_topic["주요 토픽"] = df_topic[topic_cols].idxmax(axis=1).str.extract(r'(\d+)').astype(int)

In [166]:
# 먼저 df_raw와 df_meta 병합
df_rec = pd.merge(
    df_raw[["news_id", "wdate"]],
    df_meta,
    on="news_id",
    how="left",
)

# 이어서 df_topic과 병합
df_rec = pd.merge(df_rec, df_topic[["news_id", "주요 토픽"]], on="news_id", how="left")

# 전처리
df_rec = df_rec.rename(columns={"주요 토픽": "topic"})
df_rec = df_rec.dropna().reset_index(drop=True)

In [167]:
df_rec.head(1)

,news_id,wdate,summary,stock_list,stock_list_view,industry_list,impact_score,topic
0,20250523_0002,2025-05-23 18:52:00,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33,4


In [168]:
df_rec['impact_score'].describe()

count    13749.000000
mean         0.114205
std          0.097423
min          0.000000
25%          0.060000
50%          0.090000
75%          0.140000
max          2.180000
Name: impact_score, dtype: float64

In [169]:
# 비대칭 - 오른쪽 꼬리가 김
# 평균도 중간값보다 큼

1. risk == 0 or 1
- 낮은 변동성 또는 보유 종목 관련 뉴스에 반응하고, 관심있는 주제일때만 클릭
2. risk == 2
- 관심분야만 맞으면 클릭, Impact에 연연하지 않고, 정보 수집형, 다양한 분산투자 -> 조건 완화
3. risk == 3, 4
- 임팩트 큰 뉴스에 반응, 종목.업종 중심으로 클릭

In [170]:
import pandas as pd
import json


def generate_click_log(user_profiles, news_df, max_clicks=2000):
    logs = []
    avg_impact = news_df["impact_score"].mean()

    for uid, profile in user_profiles.items():
        topics = profile["topics"]
        stocks = set(s["stock_id"] for s in profile["memberStocks"])
        industries = set(i["industry_id"] for i in profile["memberIndustries"])
        risk = profile["investScore"]
        click_count = 0

        for _, row in news_df.iterrows():
            try:
                stock_list = (
                    json.loads(row["stock_list"])
                    if isinstance(row["stock_list"], str)
                    else row["stock_list"]
                )
                industry_list = (
                    json.loads(row["industry_list"])
                    if isinstance(row["industry_list"], str)
                    else row["industry_list"]
                )
            except Exception as e:
                print(f"[Error] JSON parsing error: {e}")
                continue

            topic_match = row["topic"] in topics
            stock_match = any(
                s.get("stock_id") in stocks for s in stock_list if isinstance(s, dict)
            )
            industry_match = any(
                i.get("industry_id") in industries
                for i in industry_list
                if isinstance(i, dict)
            )
            impact = row["impact_score"]

            # 투자 성향에 따른 impact 조건
            #if risk == 0:  # 안정형
            #    impact_match = impact <= 0.06
            #elif risk == 1:  # 안정 추구형
            #    impact_match = impact <= 0.09
            #elif risk == 2:  # 위험 중립형
            #    impact_match = True  # 전부 고려
            #elif risk == 3:  # 적극 투자형
            #    impact_match = impact >= 0.09
            #elif risk == 4:  # 공격 투자형
            #    impact_match = impact >= 0.14


            if risk in [0, 1]:  # 안정형, 안정 추구형
                clicked = int(((impact <= 0.07 or stock_match or industry_match) and topic_match))

            elif risk == 2:  # 위험 중립형
                clicked = int(topic_match or stock_match or industry_match)

            elif risk in [3, 4]:  # 적극/공격 투자형
                clicked = int((impact >= 0.08) and (stock_match or industry_match or topic_match))

            logs.append(
                {
                    "user_id": uid,
                    "news_id": row["news_id"],
                    "clicked": clicked,
                    "wdate": row["wdate"],
                    "stock_list": stock_list,
                    "industry_list": industry_list,
                    "impact_score": impact,
                    "topic": row["topic"],
                }
            )

            if clicked:
                click_count += 1
            if click_count >= max_clicks:
                break  # ✅ 누적 클릭 수 제한
    return pd.DataFrame(logs)

In [171]:
df_logs = generate_click_log(user_profiles, df_rec)
df_logs

,user_id,news_id,clicked,wdate,stock_list,industry_list,impact_score,topic
0,u1,20250523_0002,0,2025-05-23 18:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4
1,u1,20250523_0004,0,2025-05-23 18:33:00,"[{'stock_id': '005930', 'stock_name': '삼성전자'}]","[{'stock_id': '005930', 'industry_id': '32604'...",0.05,9
2,u1,20250523_0007,0,2025-05-23 18:00:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4
3,u1,20250523_0010,0,2025-05-23 17:52:00,"[{'stock_id': '298040', 'stock_name': '효성중공업'}]","[{'stock_id': '298040', 'industry_id': '32801'...",0.08,9
4,u1,20250523_0011,0,2025-05-23 17:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4
...,...,...,...,...,...,...,...,...
188908,u20,20240906_0173,1,2024-09-06 08:23:00,"[{'stock_id': '030200', 'stock_name': 'KT'}]","[{'stock_id': '030200', 'industry_id': '106102...",0.08,7
188909,u20,20240906_0175,0,2024-09-06 08:22:00,"[{'stock_id': '005940', 'stock_name': 'NH투자증권'...","[{'stock_id': '005940', 'industry_id': '116601...",0.10,4
188910,u20,20240906_0183,0,2024-09-06 08:16:00,"[{'stock_id': '005940', 'stock_name': 'NH투자증권'}]","[{'stock_id': '005940', 'industry_id': '116601...",0.05,4
188911,u20,20240906_0185,1,2024-09-06 08:15:00,"[{'stock_id': '267260', 'stock_name': 'HD현대일렉트...","[{'stock_id': '267260', 'industry_id': '32801'...",0.25,2


In [172]:
# 클래스별 개수
class_counts = df_logs['clicked'].value_counts()
print("클래스별 개수:\n", class_counts)

# 클래스별 비율
class_ratio = df_logs['clicked'].value_counts(normalize=True)
print("클래스별 비율:\n", class_ratio)

클래스별 개수:
 clicked
0    151449
1     37464
Name: count, dtype: int64
클래스별 비율:
 clicked
0    0.801686
1    0.198314
Name: proportion, dtype: float64


In [173]:
df_logs.to_csv('click_logs.csv', index=False)

In [109]:
df_logs["user_id"].value_counts()

user_id
u11    13749
u5     13749
u14    13749
u13    13749
u1     13382
u4     12163
u9     10715
u17     9956
u10     9567
u6      9303
u18     8511
u16     8246
u7      8170
u12     8124
u2      7803
u19     7586
u20     6677
u15     5389
u8      4885
u3      3440
Name: count, dtype: int64

### generate_samples_strict 함수
1. 유저별로 클릭 로그 정렬
2. 과거 클릭 이력이 있는 시점에서만 샘플 생성
3. positive 후보 뉴스는
    - 실제 클릭함
    - 조건 만족
4. negative는 같은 날짜 뉴스 중
    - positive 아닌 것
    - 조건 불만족

In [110]:
from tqdm import tqdm
import pandas as pd
import random

# negative sampling 기준 완화 (2 out of 4)
def is_negative(row, topics, stock_ids, industry_ids, risk):
    topic_miss = row["topic"] not in topics
    stock_miss = not any(s["stock_id"] in stock_ids for s in row["stock_list"])
    industry_miss = not any(i["industry_id"] in industry_ids for i in row["industry_list"])
    impact_miss = (
        (row["impact_score"] > 0.06 if risk in [0, 1]
         else (row["impact_score"] < 0.09 if risk in [3, 4]
               else False))
    )
    mismatch_count = sum([topic_miss, stock_miss, industry_miss, impact_miss])
    return mismatch_count >= 2


def generate_samples_strict(
    df_log, user_profiles, max_history=10, num_negatives=10, max_samples_per_user=1000
):
    df_log["wdate"] = pd.to_datetime(df_log["wdate"])
    df_log = df_log.sort_values(by=["user_id", "wdate"])
    samples = []

    avg_impact = df_log["impact_score"].mean()
    grouped = df_log.groupby("user_id")

    for uid, group in tqdm(grouped, desc="Generating Samples"):
        if uid not in user_profiles:
            continue

        profile = user_profiles[uid]
        topics = profile["topics"]
        stock_ids = set(s["stock_id"] for s in profile["memberStocks"])
        industry_ids = set(i["industry_id"] for i in profile["memberIndustries"])
        risk = profile["investScore"]
        user_sample_count = 0

        for i, row in group.iterrows():
            if user_sample_count >= max_samples_per_user:
                break

            ts = row["wdate"]
            date_only = ts.date()
            nid = row["news_id"]

            history_df = group[group["wdate"] < ts]
            history_clicked = history_df[history_df["clicked"] == 1]["news_id"].tolist()
            if len(history_clicked) == 0:
                continue

            topic_ok = row["topic"] in topics
            stock_ok = any(s["stock_id"] in stock_ids for s in row["stock_list"])
            industry_ok = any(
                i["industry_id"] in industry_ids for i in row["industry_list"]
            )

            impact = row["impact_score"]
            if risk in [0, 1]:
                is_impact = (impact <= 0.07 or stock_ok or industry_ok) and topic_ok
            elif risk == 2:
                is_impact = topic_ok or stock_ok or industry_ok
            elif risk in [3, 4]:
                is_impact = (impact >= 0.08) and (stock_ok or industry_ok or topic_ok)


            if row["clicked"] == 1 and is_impact:
                pos = nid

                same_day = group[group["wdate"].dt.date == date_only]
                neg_pool = same_day[
                    (same_day["news_id"] != nid)
                    & same_day.apply(
                        lambda row: is_negative(row, topics, stock_ids, industry_ids, risk),
                        axis=1
                    )
                ]

                if len(neg_pool) < num_negatives:
                    continue

                negs = random.sample(neg_pool["news_id"].tolist(), num_negatives)

                samples.append(
                    {
                        "user_id": uid,
                        "clicked_history": history_clicked[-max_history:],
                        "positive": pos,
                        "negatives": negs,
                    }
                )
                user_sample_count += 1

    return pd.DataFrame(samples)

In [111]:
dataset = generate_samples_strict(df_logs, user_profiles)

Generating Samples: 100%|██████████| 20/20 [01:57<00:00,  5.85s/it]


In [112]:
dataset

,user_id,clicked_history,positive,negatives
0,u1,[20240112_0228],20240112_0139,"[20240112_0073, 20240112_0215, 20240112_0223, ..."
1,u1,"[20240112_0228, 20240112_0139]",20240112_0065,"[20240112_0221, 20240112_0137, 20240112_0151, ..."
2,u1,"[20240112_0228, 20240112_0139, 20240112_0065]",20240115_0257,"[20240115_0321, 20240115_0036, 20240115_0251, ..."
3,u1,"[20240112_0228, 20240112_0139, 20240112_0065]",20240115_0258,"[20240115_0327, 20240115_0261, 20240115_0141, ..."
4,u1,"[20240112_0228, 20240112_0139, 20240112_0065, ...",20240115_0251,"[20240115_0287, 20240115_0297, 20240115_0179, ..."
...,...,...,...,...
19995,u9,"[20241030_0316, 20241030_0293, 20241030_0159, ...",20241101_0253,"[20241101_0044, 20241101_0057, 20241101_0087, ..."
19996,u9,"[20241030_0293, 20241030_0159, 20241030_0129, ...",20241101_0236,"[20241101_0179, 20241101_0293, 20241101_0131, ..."
19997,u9,"[20241030_0159, 20241030_0129, 20241031_0373, ...",20241101_0225,"[20241101_0137, 20241101_0087, 20241101_0157, ..."
19998,u9,"[20241030_0129, 20241031_0373, 20241031_0359, ...",20241101_0103,"[20241101_0143, 20241101_0191, 20241101_0305, ..."


# 임베딩

In [113]:
from sentence_transformers import SentenceTransformer
import torch

model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"

# GPU가 있으면 GPU, 없으면 CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로딩 후 디바이스 설정
model_emb = SentenceTransformer(model_name, device=device)

In [114]:
summary_embedding = model_emb.encode(df_rec["summary"].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 430/430 [07:36<00:00,  1.06s/it]


In [115]:
df_rec["embedding"] = summary_embedding.tolist()

In [116]:
df_rec.to_csv('test.csv', index=False)

In [117]:
df_rec.head()

,news_id,wdate,summary,stock_list,stock_list_view,industry_list,impact_score,topic,embedding
0,20250523_0002,2025-05-23 18:52:00,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33,4,"[-0.2188965380191803, -1.4778021574020386, 0.3..."
1,20250523_0004,2025-05-23 18:33:00,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,"[{""stock_id"": ""005930"", ""stock_name"": ""삼성전자""}]","[{""stock_id"": ""005930"", ""stock_name"": ""삼성전자""}]","[{""stock_id"": ""005930"", ""industry_id"": ""32604""...",0.05,9,"[-0.27986857295036316, -0.5640881061553955, -0..."
2,20250523_0007,2025-05-23 18:00:00,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33,4,"[-0.5892680883407593, -1.0036866664886475, 0.3..."
3,20250523_0010,2025-05-23 17:52:00,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,"[{""stock_id"": ""298040"", ""stock_name"": ""효성중공업""}]","[{""stock_id"": ""004800"", ""stock_name"": ""효성""}, {...","[{""stock_id"": ""298040"", ""industry_id"": ""32801""...",0.08,9,"[-0.32625988125801086, -0.4966648817062378, -0..."
4,20250523_0011,2025-05-23 17:52:00,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33,4,"[-0.771115243434906, -1.1746798753738403, -0.2..."


In [118]:
df_emb = df_rec[['news_id', 'embedding']]
df_emb.head()

,news_id,embedding
0,20250523_0002,"[-0.2188965380191803, -1.4778021574020386, 0.3..."
1,20250523_0004,"[-0.27986857295036316, -0.5640881061553955, -0..."
2,20250523_0007,"[-0.5892680883407593, -1.0036866664886475, 0.3..."
3,20250523_0010,"[-0.32625988125801086, -0.4966648817062378, -0..."
4,20250523_0011,"[-0.771115243434906, -1.1746798753738403, -0.2..."


In [119]:
import ast

df_emb["embedding"] = df_emb["embedding"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_93066/192005513.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["embedding"] = df_emb["embedding"].apply(


# 모델 학습

In [120]:
import numpy as np

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [121]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split

In [122]:
from torch.nn.functional import normalize


class NewsRecDataset(Dataset):
    def __init__(self, samples, news_embs):
        self.samples = samples
        self.news_embs = news_embs

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        row = self.samples[idx]

        # 정규화 적용된 벡터
        clicked_mean = normalize(
            torch.stack([self.news_embs[nid] for nid in row["clicked_history"]]).mean(
                dim=0
            ),
            dim=0,
        )
        pos_vec = normalize(self.news_embs[row["positive"]], dim=0)
        neg_vecs = normalize(
            torch.stack([self.news_embs[nid] for nid in row["negatives"]]), dim=1
        )

        return clicked_mean, pos_vec, neg_vecs

In [123]:
class TwoTowerRecommender(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128):
        super().__init__()
        self.user_tower = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
        )
        self.news_tower = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
        )

    def forward(self, clicked_embs, candidate_embs):
        # clicked_embs: [B, N, D] → 평균 → [B, D]
        user_vec = clicked_embs
        user_vec = F.normalize(self.user_tower(user_vec), dim=1)  # [B, H]

        # candidate_embs: [B, K+1, D]
        item_vecs = F.normalize(self.news_tower(candidate_embs), dim=2)  # [B, K+1, H]

        # 유사도 계산: [B, 1, H] · [B, H, K+1] → [B, K+1]
        scores = torch.matmul(user_vec.unsqueeze(1), item_vecs.transpose(1, 2)).squeeze(
            1
        )
        return scores

In [124]:
# 🔹 손실 함수: InfoNCE (positive는 항상 index 0)
def contrastive_loss(scores):
    labels = torch.zeros(scores.size(0), dtype=torch.long)
    return F.cross_entropy(scores, labels)

# 🔹 평가 지표: Top-1 Accuracy, MRR, Recall@K
def evaluate(model, loader, k=3):
    model.eval()
    top1_correct, mrr_total, recall_total, total = 0, 0, 0, 0

    with torch.no_grad():
        for clicked, pos, negs in loader:
            candidates = torch.cat([pos.unsqueeze(1), negs], dim=1)
            scores = model(clicked, candidates)

            # Top-1 Accuracy
            labels = torch.zeros(scores.size(0), dtype=torch.long)  # 정답 인덱스가 항상 0인 경우

            top1_correct += (scores.argmax(dim=1) == labels).sum().item()

            # MRR 계산
            ranks = (scores.argsort(dim=1, descending=True) == labels.unsqueeze(1)).nonzero(as_tuple=False)
            mrr_total += sum([1.0 / (r.item() + 1) for r in ranks[:, 1]])

            # Recall@K
            topk = scores.topk(k, dim=1).indices
            recall_total += (topk == labels.unsqueeze(1)).any(dim=1).sum().item()
            total += clicked.size(0)

    print(f"[Eval] Top-1 Acc: {top1_correct/total:.4f} | MRR: {mrr_total/total:.4f} | Recall@{k}: {recall_total}")


In [125]:
import torch

# news_embedding: DataFrame → Dict[str, Tensor]
news_embeddings = {
    row["news_id"]: torch.tensor(row["embedding"], dtype=torch.float32)
    for _, row in df_emb.iterrows()
}

# samples: DataFrame → List[Dict]
sample_data = []
for _, row in dataset.iterrows():
    sample_data.append(
        {
            "user_id": row["user_id"],
            "clicked_history": row["clicked_history"],
            "positive": row["positive"],
            "negatives": row["negatives"],
        }
    )

In [126]:
class EarlyStopping:
    def __init__(self, patience=3, verbose=True):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.verbose = verbose

    def __call__(self, val_loss):
        score = -val_loss  # loss가 낮을수록 좋은 것이므로 부호 반전

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f"⏳ EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

In [127]:
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 🔹 학습/검증 분리
train_data, val_data = train_test_split(sample_data, test_size=0.3, random_state=42)

train_loader = DataLoader(
    NewsRecDataset(train_data, news_embeddings), batch_size=32, shuffle=True
)
val_loader = DataLoader(
    NewsRecDataset(val_data, news_embeddings), batch_size=32, shuffle=False
)

# 🔹 모델 초기화
model = TwoTowerRecommender()
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

# 🔹 스케줄러: val_loss가 줄지 않으면 LR 감소
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=1, min_lr=1e-5
)


# 🔹 학습 루프
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 🔹 학습 루프 (디버깅 추가)
early_stopper = EarlyStopping(patience=3, verbose=True)

for epoch in range(20):  # 더 길게 잡아도 early stopping이 막아줌
    model.train()
    total_loss = 0

    for batch_idx, (clicked, pos, negs) in enumerate(train_loader):
        try:
            candidates = torch.cat([pos.unsqueeze(1), negs], dim=1)
            scores = model(clicked, candidates)
            loss = contrastive_loss(scores)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        except Exception as e:
            print(f"[❌ Error in Batch {batch_idx}]: {type(e).__name__}: {e}")

    avg_train_loss = total_loss / len(train_loader)
    print(f"\n✅ [Epoch {epoch+1}] Avg Train Loss: {avg_train_loss:.4f}")

    # 🔹 검증 루프 → loss 계산
    model.eval()
    val_total_loss = 0
    with torch.no_grad():
        for clicked, pos, negs in val_loader:
            candidates = torch.cat([pos.unsqueeze(1), negs], dim=1)
            scores = model(clicked, candidates)
            val_loss = contrastive_loss(scores)
            val_total_loss += val_loss.item()
    avg_val_loss = val_total_loss / len(val_loader)
    print(f"[Eval] Val Loss: {avg_val_loss:.4f}")

    # 🔹 스케줄러 및 EarlyStopping 적용
    scheduler.step(avg_val_loss)
    early_stopper(avg_val_loss)
    if early_stopper.early_stop:
        print("🛑 Early stopping triggered!")
        break

    # 🔹 평가 지표 출력
    evaluate(model, val_loader)


✅ [Epoch 1] Avg Train Loss: 2.2534
[Eval] Val Loss: 2.2014
[Eval] Top-1 Acc: 0.2278 | MRR: 0.4317 | Recall@3: 3088

✅ [Epoch 2] Avg Train Loss: 2.1676
[Eval] Val Loss: 2.1767
[Eval] Top-1 Acc: 0.2333 | MRR: 0.4416 | Recall@3: 3233

✅ [Epoch 3] Avg Train Loss: 2.1331
[Eval] Val Loss: 2.1633
[Eval] Top-1 Acc: 0.2393 | MRR: 0.4482 | Recall@3: 3302

✅ [Epoch 4] Avg Train Loss: 2.1066
[Eval] Val Loss: 2.1597
[Eval] Top-1 Acc: 0.2385 | MRR: 0.4481 | Recall@3: 3290

✅ [Epoch 5] Avg Train Loss: 2.0832
[Eval] Val Loss: 2.1561
[Eval] Top-1 Acc: 0.2392 | MRR: 0.4501 | Recall@3: 3319

✅ [Epoch 6] Avg Train Loss: 2.0622
[Eval] Val Loss: 2.1485
[Eval] Top-1 Acc: 0.2397 | MRR: 0.4515 | Recall@3: 3333

✅ [Epoch 7] Avg Train Loss: 2.0440
[Eval] Val Loss: 2.1475
[Eval] Top-1 Acc: 0.2420 | MRR: 0.4539 | Recall@3: 3365

✅ [Epoch 8] Avg Train Loss: 2.0242
[Eval] Val Loss: 2.1401
[Eval] Top-1 Acc: 0.2493 | MRR: 0.4587 | Recall@3: 3373

✅ [Epoch 9] Avg Train Loss: 2.0074
[Eval] Val Loss: 2.1437
⏳ EarlyStopp

# 모델 저장

In [128]:
pip install onnx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [129]:
import torch

# 예시 입력 (1명의 사용자, 클릭 5개, 후보 10개, 임베딩 dim=768)
dummy_clicked = torch.randn(1, 5, 768)  # [B, N, D]
dummy_candidates = torch.randn(1, 10, 768)  # [B, K, D]

# 모델 저장
torch.onnx.export(
    model,
    (dummy_clicked, dummy_candidates),
    "two_tower_model.onnx",
    input_names=["clicked", "candidates"],
    output_names=["scores"],
    dynamic_axes={
        "clicked": {1: "click_seq_len"},  # 클릭 히스토리 길이
        "candidates": {1: "num_candidates"},  # 후보 개수
    },
    opset_version=11,
)
print("✅ ONNX 모델 저장 완료")

✅ ONNX 모델 저장 완료


In [130]:
pip install onnxruntime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [131]:
import onnxruntime as ort
import numpy as np

# ONNX 세션 생성
session = ort.InferenceSession("two_tower_model.onnx")

# 테스트용 데이터 (1명의 사용자, 클릭 3개, 후보 5개)
clicked_np = np.random.randn(1, 3, 768).astype(np.float32)
candidates_np = np.random.randn(1, 20, 768).astype(np.float32)

# 추론
outputs = session.run(None, {"clicked": clicked_np, "candidates": candidates_np})

# 결과
scores = outputs[0]  # [1, 5]
top_k = scores[0].argsort()[::-1][:1]  # Top-3 추천 인덱스
print("✅ 예측 결과 (Top-3 후보 인덱스):\n", top_k[0])

✅ 예측 결과 (Top-3 후보 인덱스):
 [ 9  3  6 19  5  4  8 10 17  2 15  7 12 11 13  0 16 14 18  1]


In [132]:
clicked_np

array([[[ 0.49671414, -0.1382643 ,  0.64768857, ..., -0.70317644,
         -0.03498849,  1.7708006 ],
        [-0.6269671 ,  1.8124485 ,  0.7077519 , ..., -0.14760259,
         -0.4660365 , -1.5947031 ],
        [ 0.5136001 , -0.53270084, -1.1699169 , ..., -0.99806064,
         -0.38397127,  0.2502002 ]]], dtype=float32)

## ML 데이터셋 

In [133]:
df_users = pd.DataFrame(
    [
        {
            "user_id": user_id,
            "userPnl": profile["userPnl"],
            "asset": profile["asset"],
            "investScore": profile["investScore"],
            "topics": profile["topics"],
            "memberStocks": profile["memberStocks"],
            "memberIndustries": profile["memberIndustries"],
        }
        for user_id, profile in user_profiles.items()
    ]
)

In [134]:
df_users.head()

,user_id,userPnl,asset,investScore,topics,memberStocks,memberIndustries
0,u1,-220000,7000000,4,"[1, 6]","[{'stock_id': '019170', 'stock_name': '신풍제약'},...","[{'stock_id': '000100', 'industry_id': '32102'..."
1,u2,37000,2500000,1,"[2, 7, 8]","[{'stock_id': '005930', 'stock_name': '삼성전자'},...","[{'stock_id': '005930', 'industry_id': '32604'..."
2,u3,420000,15000000,2,"[1, 3, 4]","[{'stock_id': '012450', 'stock_name': '한화에어로스페...","[{'stock_id': '012450', 'industry_id': '33103'..."
3,u4,50000,1000000,1,"[2, 3, 5]","[{'stock_id': '005380', 'stock_name': '현대차'}, ...","[{'stock_id': '005380', 'industry_id': '33001'..."
4,u5,12000,500000,0,"[7, 9]","[{'stock_id': '000660', 'stock_name': 'SK하이닉스'}]","[{'stock_id': '000660', 'industry_id': '32601'..."


In [135]:
df_logs.head()

,user_id,news_id,clicked,wdate,stock_list,industry_list,impact_score,topic
0,u1,20250523_0002,0,2025-05-23 18:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4
1,u1,20250523_0004,0,2025-05-23 18:33:00,"[{'stock_id': '005930', 'stock_name': '삼성전자'}]","[{'stock_id': '005930', 'industry_id': '32604'...",0.05,9
2,u1,20250523_0007,0,2025-05-23 18:00:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4
3,u1,20250523_0010,0,2025-05-23 17:52:00,"[{'stock_id': '298040', 'stock_name': '효성중공업'}]","[{'stock_id': '298040', 'industry_id': '32801'...",0.08,9
4,u1,20250523_0011,0,2025-05-23 17:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4


In [136]:
df_ml_dataset = pd.merge(df_logs, df_users, on=["user_id"], how="left")

In [137]:
df_topic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   news_id  14087 non-null  object 
 1   topic_1  14087 non-null  float64
 2   topic_2  14087 non-null  float64
 3   topic_3  14087 non-null  float64
 4   topic_4  14087 non-null  float64
 5   topic_5  14087 non-null  float64
 6   topic_6  14087 non-null  float64
 7   topic_7  14087 non-null  float64
 8   topic_8  14087 non-null  float64
 9   topic_9  14087 non-null  float64
 10  주요 토픽    14087 non-null  int64  
dtypes: float64(9), int64(1), object(1)
memory usage: 1.2+ MB


In [138]:
# 붙일 컬럼만 추출
topic_cols = ['news_id'] + [f'topic_{i}' for i in range(1, 10)]
df_topic_subset = df_topic[topic_cols].drop_duplicates(subset='news_id')

# 병합
df_ml_dataset = df_ml_dataset.merge(df_topic_subset, on='news_id', how='left')

In [139]:
df_ml_dataset.head()

,user_id,news_id,clicked,wdate,stock_list,industry_list,impact_score,topic,userPnl,asset,...,memberIndustries,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,u1,20250523_0002,0,2025-05-23 18:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4,-220000,7000000,...,"[{'stock_id': '000100', 'industry_id': '32102'...",0.0122,0.0119,0.0121,0.5421,0.3730,0.0122,0.0120,0.0122,0.0122
1,u1,20250523_0004,0,2025-05-23 18:33:00,"[{'stock_id': '005930', 'stock_name': '삼성전자'}]","[{'stock_id': '005930', 'industry_id': '32604'...",0.05,9,-220000,7000000,...,"[{'stock_id': '000100', 'industry_id': '32102'...",0.0097,0.0096,0.0097,0.1966,0.0104,0.2332,0.0098,0.1834,0.3376
2,u1,20250523_0007,0,2025-05-23 18:00:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4,-220000,7000000,...,"[{'stock_id': '000100', 'industry_id': '32102'...",0.0114,0.0114,0.0116,0.5937,0.3260,0.0114,0.0113,0.0115,0.0118
3,u1,20250523_0010,0,2025-05-23 17:52:00,"[{'stock_id': '298040', 'stock_name': '효성중공업'}]","[{'stock_id': '298040', 'industry_id': '32801'...",0.08,9,-220000,7000000,...,"[{'stock_id': '000100', 'industry_id': '32102'...",0.0146,0.0145,0.0147,0.1025,0.1528,0.0145,0.0945,0.0147,0.5772
4,u1,20250523_0011,0,2025-05-23 17:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4,-220000,7000000,...,"[{'stock_id': '000100', 'industry_id': '32102'...",0.1741,0.0130,0.0969,0.4690,0.0131,0.0132,0.0128,0.1948,0.0130


In [143]:
# topic_1 ~ topic_9 컬럼 이름 리스트
topic_cols = [f"topic_{i}" for i in range(1, 10)]

# 각 행에서 가장 큰 topic 컬럼의 이름 추출 → 숫자만 남김
df_ml_dataset["topic"] = df_ml_dataset[topic_cols].idxmax(axis=1).str.extract("(\d+)").astype(int)


In [144]:
df_ml_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188913 entries, 0 to 188912
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           188913 non-null  object        
 1   news_id           188913 non-null  object        
 2   clicked           188913 non-null  int64         
 3   wdate             188913 non-null  datetime64[ns]
 4   stock_list        188913 non-null  object        
 5   industry_list     188913 non-null  object        
 6   impact_score      188913 non-null  float64       
 7   topic             188913 non-null  int64         
 8   userPnl           188913 non-null  int64         
 9   asset             188913 non-null  int64         
 10  investScore       188913 non-null  int64         
 11  topics            188913 non-null  object        
 12  memberStocks      188913 non-null  object        
 13  memberIndustries  188913 non-null  object        
 14  topi

In [147]:
# is_same_stock, is_same_industry 계산 함수
def check_overlap(news_list, user_list, key):
    news_ids = {item[key] for item in news_list}
    user_ids = {item[key] for item in user_list}
    return int(len(news_ids & user_ids) > 0)


# 적용
df_ml_dataset["is_same_stock"] = df_ml_dataset.apply(
    lambda row: check_overlap(row["stock_list"], row["memberStocks"], "stock_id"),
    axis=1,
)
df_ml_dataset["is_same_industry"] = df_ml_dataset.apply(
    lambda row: check_overlap(
        row["industry_list"], row["memberIndustries"], "industry_id"
    ),
    axis=1,
)

In [148]:
df_ml_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188913 entries, 0 to 188912
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           188913 non-null  object        
 1   news_id           188913 non-null  object        
 2   clicked           188913 non-null  int64         
 3   wdate             188913 non-null  datetime64[ns]
 4   stock_list        188913 non-null  object        
 5   industry_list     188913 non-null  object        
 6   impact_score      188913 non-null  float64       
 7   topic             188913 non-null  int64         
 8   userPnl           188913 non-null  int64         
 9   asset             188913 non-null  int64         
 10  investScore       188913 non-null  int64         
 11  topics            188913 non-null  object        
 12  memberStocks      188913 non-null  object        
 13  memberIndustries  188913 non-null  object        
 14  topi

In [150]:
# 필요한 컬럼 리스트
columns_to_keep = [
    'userPnl', 'asset', 'investScore',
    'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
    'topic_6', 'topic_7', 'topic_8', 'topic_9',
    'topic',  # → main_topic으로 바꿀 예정
    'is_same_stock', 'is_same_industry',
    'clicked'
]

# 필요한 컬럼만 남기기
df_ml_final = df_ml_dataset[columns_to_keep].copy()

# 컬럼명 변경
df_ml_final.rename(columns={"topic": "main_topic"}, inplace=True)

In [151]:
df_ml_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188913 entries, 0 to 188912
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   userPnl           188913 non-null  int64  
 1   asset             188913 non-null  int64  
 2   investScore       188913 non-null  int64  
 3   topic_1           188913 non-null  float64
 4   topic_2           188913 non-null  float64
 5   topic_3           188913 non-null  float64
 6   topic_4           188913 non-null  float64
 7   topic_5           188913 non-null  float64
 8   topic_6           188913 non-null  float64
 9   topic_7           188913 non-null  float64
 10  topic_8           188913 non-null  float64
 11  topic_9           188913 non-null  float64
 12  main_topic        188913 non-null  int64  
 13  is_same_stock     188913 non-null  int64  
 14  is_same_industry  188913 non-null  int64  
 15  clicked           188913 non-null  int64  
dtypes: float64(9), int64

In [152]:
df_ml_final.to_csv('./recommend_ml_dataset_v2.csv')
df_ml_final.head()

,userPnl,asset,investScore,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,main_topic,is_same_stock,is_same_industry,clicked
0,-220000,7000000,4,0.0122,0.0119,0.0121,0.5421,0.3730,0.0122,0.0120,0.0122,0.0122,4,0,0,0
1,-220000,7000000,4,0.0097,0.0096,0.0097,0.1966,0.0104,0.2332,0.0098,0.1834,0.3376,9,0,0,0
2,-220000,7000000,4,0.0114,0.0114,0.0116,0.5937,0.3260,0.0114,0.0113,0.0115,0.0118,4,0,0,0
3,-220000,7000000,4,0.0146,0.0145,0.0147,0.1025,0.1528,0.0145,0.0945,0.0147,0.5772,9,0,0,0
4,-220000,7000000,4,0.1741,0.0130,0.0969,0.4690,0.0131,0.0132,0.0128,0.1948,0.0130,4,0,0,0
